In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from scipy.ndimage.filters import gaussian_filter
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns
%matplotlib inline


spark = SparkSession.builder\
    .appName("mongo")\
    .getOrCreate()



In [9]:
fp = spark.read.csv('/Users/vvviren/MyDev/bd apps/Film_Permits.csv',header='true')

In [10]:
cols = [
 'EventType',
 'StartDateTime',
 'EndDateTime',
 'EnteredOn',
 'EventAgency',
 'ParkingHeld',
 'Borough',
 'CommunityBoard(s)',
 'PolicePrecinct(s)',
 'Category',
 'SubCategoryName',
 'Country']
fp=fp.drop(*cols)

In [11]:
import pyspark.sql.functions as f


fp=fp.withColumn('tmp', f.split('ZipCode(s)', ', '))                                               
fp=fp.withColumn('zips', f.explode('tmp')).drop("tmp", "ZipCode(s)", "EventID")


fp=fp.filter(f.col("zips")>=10001)

fp = fp.groupby("zips").count()



In [12]:
fp.show()

+-----+-----+
| zips|count|
+-----+-----+
|11205|  949|
|10309|  131|
|11236|   16|
|11106| 1585|
|11251|   99|
|10110|   15|
|11218|  456|
|10452|  165|
|11428|    3|
|11237| 1194|
|10177|   33|
|11379|  107|
|11364|   41|
|11109|  135|
|11249| 1572|
|10012| 1978|
|11001|    5|
|11238|  687|
|11385| 1171|
|10039|   80|
+-----+-----+
only showing top 20 rows



In [14]:
geo = spark.read.csv('/Users/vvviren/MyDev/bd apps/nycziptags.csv',header='true')

geo.show()


+-------+----------+-----------+
|zipcode|       lat|        lon|
+-------+----------+-----------+
|  10001| 40.750422| -73.996328|
|  10002| 40.718758| -73.986427|
|  10003| 40.730223| -73.988564|
|  10004|40.7038704|-74.0160428|
|  10005| 40.706903| -74.008654|
|  10006| 40.708834| -74.013168|
|  10007| 40.713941| -74.007401|
|  10009|  40.72775| -73.980396|
|  10010| 40.739024| -73.983542|
|  10011| 40.740916| -73.999769|
|  10012| 40.729124| -73.991582|
|  10013| 40.722105| -74.003497|
|  10014| 40.738088| -74.005095|
|  10016| 40.745527| -73.978449|
|  10017| 40.752955| -73.973196|
|  10018| 40.755332| -73.993172|
|  10019| 40.765926| -73.985443|
|  10020| 40.735449| -73.996788|
|  10021| 40.768823| -73.960257|
|  10022| 40.758775| -73.967842|
+-------+----------+-----------+
only showing top 20 rows



In [15]:
geo.printSchema()

root
 |-- zipcode: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)



In [17]:
geo = geo.withColumn("zipcode", f.trim(f.col('zipcode')))

geo = geo.withColumn("zipcode", f.col('zipcode').cast('int'))
geo = geo.withColumn("lat", f.trim(f.col('lat')))

geo = geo.withColumn("lat", f.col('lat').cast('float'))
geo = geo.withColumn("lon", f.trim(f.col('lon')))

geo = geo.withColumn("lon", f.col('lon').cast('float'))
geo.printSchema()

root
 |-- zipcode: integer (nullable = true)
 |-- lat: float (nullable = true)
 |-- lon: float (nullable = true)



In [19]:
fp_geo = fp.join(geo, fp["zips"]==geo["zipcode"], how="inner").drop("zips")
fp_geo.show()

+-----+-------+---------+----------+
|count|zipcode|      lat|       lon|
+-----+-------+---------+----------+
|  949|  11205|40.682507| -73.93627|
|  131|  10309| 40.53177| -74.21878|
|   16|  11236| 40.65701| -73.91581|
| 1585|  11106|  40.7606|-73.930984|
|   99|  11251|  40.6451| -73.94503|
|   15|  10110|40.753986| -73.98081|
|  456|  11218| 40.64434| -73.97841|
|  165|  10452| 40.83873| -73.92161|
|    3|  11428| 40.72064|-73.742096|
| 1194|  11237| 40.70219| -73.91807|
|   33|  10177|40.755333| -73.97608|
|  107|  11379| 40.71924| -73.89279|
|   41|  11364| 40.74277| -73.75883|
|  135|  11109| 40.65138| -73.87078|
| 1572|  11249|  40.6451| -73.94503|
| 1978|  10012|40.729122|-73.991585|
|    5|  11001| 40.72005| -73.70678|
|  687|  11238| 40.68059| -73.96393|
| 1171|  11385|40.702404| -73.88598|
|   80|  10039|40.826736|  -73.9394|
+-----+-------+---------+----------+
only showing top 20 rows



In [25]:
pandas_df = fp_geo.toPandas()
pandas_df.to_json('/Users/vvviren/MyDev/bd apps/fp_geo.json', orient='records')

# fp_geo.coalesce(1).write.format('json').save('/Users/vvviren/MyDev/bd apps/ab.json')
